In [1]:
!pip3 install tensorflow-gpu
!pip3 install Pillow==6.2.1

  Using cached https://files.pythonhosted.org/packages/25/44/47f0722aea081697143fbcf5d2aa60d1aee4aaacb5869aee2b568974777b/tensorflow_gpu-2.0.0-cp36-cp36m-manylinux2010_x86_64.whl
ERROR: Operation cancelled by user


KeyboardInterrupt: ignored

In [2]:
#Install this if no gpu is detected
!pip3 install tf-nightly-gpu

     |████████████████████████████████| 441.9MB 38kB/s 
     |████████████████████████████████| 460kB 39.6MB/s 
     |████████████████████████████████| 3.9MB 45.5MB/s 
     |████████████████████████████████| 2.9MB 29.8MB/s 
     |████████████████████████████████| 81kB 10.2MB/s 
ERROR: google-colab 1.0.0 has requirement google-auth~=1.4.0, but you'll have google-auth 1.9.0 which is incompatible.
ERROR: tb-nightly 2.2.0a20191218 has requirement grpcio>=1.24.3, but you'll have grpcio 1.15.0 which is incompatible.
  Found existing installation: google-auth 1.4.2
    Uninstalling google-auth-1.4.2:
      Successfully uninstalled google-auth-1.4.2
  Found existing installation: h5py 2.8.0
    Uninstalling h5py-2.8.0:
      Successfully uninstalled h5py-2.8.0


In [8]:
!pip3 uninstall tensorflow-gpu

Uninstalling tensorflow-gpu-2.0.0:
  Would remove:
    /usr/local/lib/python3.6/dist-packages/tensorflow_gpu-2.0.0.dist-info/*
Proceed (y/n)? y
  Successfully uninstalled tensorflow-gpu-2.0.0


## Download Data

In [1]:
!wget http://images.cocodataset.org/zips/val2017.zip

--2019-12-18 15:18:03--  http://images.cocodataset.org/zips/val2017.zip
Resolving images.cocodataset.org (images.cocodataset.org)... 52.216.229.243
Connecting to images.cocodataset.org (images.cocodataset.org)|52.216.229.243|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 815585330 (778M) [application/zip]
Saving to: ‘val2017.zip’

val2017.zip         100%[===================>] 777.80M  13.6MB/s    in 60s     

2019-12-18 15:19:03 (13.0 MB/s) - ‘val2017.zip’ saved [815585330/815585330]



In [0]:
!mkdir data


In [0]:
!mv val2017.zip data

In [4]:
!cd data && ls && unzip -q val2017.zip

val2017.zip


In [9]:
!wget http://images.cocodataset.org/annotations/annotations_trainval2017.zip

--2019-12-18 15:20:39--  http://images.cocodataset.org/annotations/annotations_trainval2017.zip
Resolving images.cocodataset.org (images.cocodataset.org)... 52.217.37.92
Connecting to images.cocodataset.org (images.cocodataset.org)|52.217.37.92|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 252907541 (241M) [application/zip]
Saving to: ‘annotations_trainval2017.zip’

annotations_trainva 100%[===================>] 241.19M  17.1MB/s    in 16s     

2019-12-18 15:20:55 (15.3 MB/s) - ‘annotations_trainval2017.zip’ saved [252907541/252907541]



In [0]:
!mv annotations_trainval2017.zip data

In [11]:
!cd data && ls && unzip annotations_trainval2017.zip

annotations_trainval2017.zip  val2017
Archive:  annotations_trainval2017.zip
  inflating: annotations/instances_train2017.json  
  inflating: annotations/instances_val2017.json  
  inflating: annotations/captions_train2017.json  
  inflating: annotations/captions_val2017.json  
  inflating: annotations/person_keypoints_train2017.json  
  inflating: annotations/person_keypoints_val2017.json  


In [12]:
!cd data && ls && rm annotations_trainval2017.zip && rm val2017.zip

annotations  annotations_trainval2017.zip  val2017
rm: cannot remove 'val2017.zip': No such file or directory


In [13]:
!cd data && ls

annotations  val2017


## SETUP BEGINS HERE!

In [14]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os
import json
from PIL import Image

In [0]:
DATA_DIR = "data"

IMG_HEIGHT = 244
IMG_WIDTH = 244

SEED = 1

In [16]:
# Setup GPUs
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        print(e)


1 Physical GPUs, 1 Logical GPUs


# Loading Data

In [0]:
# Load Image data
image_filenames = {}
image_dir = DATA_DIR + "/val2017/"

for filename in os.listdir(image_dir):
    if filename.endswith(".jpg"):
        image_filenames[int(filename[:-4])] = image_dir + filename

In [0]:
# Load bbox/annotations
with open(DATA_DIR + "/annotations/instances_val2017.json") as f:
    _ = json.load(f)
    categories = {entry["id"]:{
            "supercategory": entry["supercategory"],
            "name": entry["name"]
        } for entry in _["categories"]}
    instances = {entry["id"]:{
            "bbox": tuple(entry["bbox"]),
            "category_id": entry["category_id"],
            "image_id": entry["image_id"]
        } for entry in _["annotations"]}

In [19]:
from google.colab import drive
drive.mount("/content/drive")

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


# Env setup

In [20]:
!cd drive/My\ Drive/colab_data_folder && ls

data  multiagent  requirements.txt  setup.py


In [0]:
import sys
sys.path.append("drive/My Drive/colab_data_folder")
from multiagent.infrastructure.env import ObjectLocalizationEnv, BetterRewardEnv, MovingEdgeEnv, HierarchicalZoomEnv, StretchyZoomEnv

In [22]:
model = tf.keras.models.Sequential(tf.keras.applications.VGG16(weights="imagenet", include_top = True).layers[:-2])

553467904/553467096 [==============================] - 10s 0us/step


In [0]:
og_env = ObjectLocalizationEnv(model, (224, 224), feature_dim = 4096)
moving_edge_env = MovingEdgeEnv(model, (244, 244), feature_dim = 4096, transformation_factor = 0.4)
zoomzoom_env = HierarchicalZoomEnv(model, (244,244), feature_dim = 4096)
stretchyzoom_env = StretchyZoomEnv(model, (244,244), feature_dim = 4096)

# Training

In [24]:
!cd drive/My\ Drive/colab_data_folder/multiagent/infrastructure && ls

env.py	__init__.py  __pycache__  rl_trainer2.py  rl_trainer.py  trainer.py


In [0]:
from multiagent.infrastructure.rl_trainer import RL_Trainer
from multiagent.agents.dqn_agent import DQN_Agent
from multiagent.util.dqn_utils import EpochSchedule

In [0]:
agent_params = { #TODO!!!!
    "replay_buffer_size": 2000,
    "batch_size": 32,
    "gamma": 0.1,
    "epsilon": EpochSchedule(5, final_p = .1, initial_p = 1),
    "dropout": 0.1
}

In [0]:
params = {
    "learning_freq": 1,
    "seed": SEED,
    "agent_class": DQN_Agent,
    "optimizer": tf.keras.optimizers.Adam(learning_rate = 0.001),
    "loss": tf.keras.losses.Huber(),
    "agent_params": agent_params,
    "env": None, #PlaceHolder
    "max_path_length": 200,
    "model_name": "austensux", #PlaceHolder
    "save_freq": 0 #Manually save after every epoch
}

## Curating category-specific data

In [0]:
CATEGORY_ID = 1
NUM_EPOCHS = 15

In [29]:
from multiagent.util.data import find_all_instances, find_image_ids_with_category, process_image_filename, stream_images

category_image_ids = find_image_ids_with_category(instances, CATEGORY_ID)
print("{} images that include at least one instance of '{}'".format(len(category_image_ids), categories[CATEGORY_ID]["name"]))

2693 images that include at least one instance of 'person'


In [0]:
category_images = stream_images(category_image_ids, image_filenames)

## Training Loop

In [0]:
from copy import deepcopy

In [0]:
current_agent_params = deepcopy(agent_params)

params['env'] = og_env
params['model_name'] = "caceido"
params['agent_params'] = current_agent_params

In [0]:
trainer = RL_Trainer(params=params)

In [34]:
tf.test.is_gpu_available()

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


True

In [35]:
tf.__version__

'2.1.0-dev20191218'

In [0]:
import time
start_time = time.time()

for epoch_iter in range(NUM_EPOCHS):
    
#     if epoch_iter:
#         trainer.agent.q_func = tf.keras.models.load_model("models/{}/epoch{}".format(params['model_name'], epoch_iter - 1))
    
    avg_returns = []
    avg_losses = []
    current_image_iter = 0
    for image_id, image_tensor in category_images:
        current_image_iter += 1
        print("\n\n####### Image {} of {} in Epoch {}".format(current_image_iter, len(category_image_ids), epoch_iter))
        
        keys, target_bboxs = find_all_instances(instances, image_id, CATEGORY_ID)
        image_tensor = tf.expand_dims(image_tensor, 0)
        
        trainer.env.training_reset(target_bboxs, image_tensor)
        returns, losses = trainer.run_training_loop(1, start_time = start_time, verbose = current_image_iter % 1000 == 0)
        avg_returns.append(np.average(returns))
        avg_losses.append(np.average(losses))
        
        print("{} unfound instances out of {}".format(len(trainer.env.target_bboxs), len(trainer.env.orig_target_bboxs)))
        
        
    trainer.agent.epsilon.increment_epoch()
    
    # Saving progress and metrics
    trainer.agent.q_func.save("drive/My Drive/models/{}/epoch{}/".format(params['model_name'], epoch_iter))
    np.save("drive/My Drive/models/returns{}_epoch{}.npy".format(params['model_name'], epoch_iter), avg_returns)
    np.save("drive/My Drive/models/losses{}_epoch{}.npy".format(params['model_name'], epoch_iter), avg_losses)
    
    
    



####### Image 1 of 2693 in Epoch 0
********** Iteration 0 ************
LOSS:  	 0.26612073
1 unfound instances out of 2


####### Image 2 of 2693 in Epoch 0
********** Iteration 0 ************
LOSS:  	 0.93687683
2 unfound instances out of 4


####### Image 3 of 2693 in Epoch 0
********** Iteration 0 ************
LOSS:  	 1.6783472
0 unfound instances out of 1


####### Image 4 of 2693 in Epoch 0
********** Iteration 0 ************
LOSS:  	 1.3096833
1 unfound instances out of 1


####### Image 5 of 2693 in Epoch 0
********** Iteration 0 ************
LOSS:  	 0.73359704
0 unfound instances out of 1


####### Image 6 of 2693 in Epoch 0
********** Iteration 0 ************
LOSS:  	 0.81859714
0 unfound instances out of 1


####### Image 7 of 2693 in Epoch 0
********** Iteration 0 ************
LOSS:  	 0.70619845
0 unfound instances out of 2


####### Image 8 of 2693 in Epoch 0
********** Iteration 0 ************
LOSS:  	 0.6116785
2 unfound instances out of 4


####### Image 9 of 2693 i

In [27]:
trainer.agent.epsilon.increment_epoch()
trainer.agent.q_func.save("drive/My Drive/models/{}/epoch{}/".format(params['model_name'], epoch_iter))
np.save("drive/My Drive/models/returns{}_epoch{}.npy".format(params['model_name'], epoch_iter), avg_returns)
np.save("drive/My Drive/models/losses{}_epoch{}.npy".format(params['model_name'], epoch_iter), avg_losses)


INFO:tensorflow:Assets written to: drive/My Drive/models/caceido/epoch0/assets


INFO:tensorflow:Assets written to: drive/My Drive/models/caceido/epoch0/assets


In [0]:
image_tensor

In [0]:
a, b = category_images.__next__()

In [0]:
d,e,f = b.shape